# Deploy LLama2 13B Chat LMI Model on AWS Inferentia


In this notebook, we explore how to host a LLama2 13B Chat large language model on SageMaker using the DeepSpeed. We use DJLServing as the model serving solution in this example that is bundled in the Large Model Inference (LMI) container. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent blog post (https://aws.amazon.com/blogs/machine-learning/deploy-bloom-176b-and-opt-30b-on-amazon-sagemaker-with-large-model-inference-deep-learning-containers-and-deepspeed/).


Model parallelism can help deploy large models that would normally be too large for a single GPU. With model parallelism, we partition and distribute a model across multiple GPUs. Each GPU holds a different part of the model, resolving the memory capacity issue for the largest deep learning models with billions of parameters. 

SageMaker has rolled out DeepSpeed container which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we deploy `'meta-llama/Llama-2-13b-chat-hf` model on a `ml.g5.12xlarge` instance. 

## Prerequisite
### Hugging Face Account

You need to have Hugging Face account. Sign Up here https://huggingface.co/join with your email if you do not already have account.

- For seamless access of the models avaialble on Hugging Face especially gated models such as Llama, for fine-tuning and inferencing purposes, you need to have Hugging Face Account to obtain read Access Token.
- After signup, [login](https://huggingface.co/login) to visit https://huggingface.co/settings/tokens to create read Access token.

### Request access to the next version of Llama

Use the same email id to obtain permission from meta by visiting this link: https://ai.meta.com/resources/models-and-libraries/llama-downloads/

- The Llama models available via Hugging Face are gated models. The use of Llama model is governed by the Meta license. In order to download the model weights and tokenizer, please visit https://ai.meta.com/resources/models-and-libraries/llama-downloads/ and accept their License before requesting access.
- Within 2 days you might be granted access to use Llama models via a confirmation email with subject: [Access granted] Your request to access model meta-llama/Llama-2-13b-chat-hf has been accepted. Though the model id is Llama-2-13b-chat-hf, you should be able to access other variants too.


In [ ]:
!pip install -Uq pip
!pip install -Uq sagemaker boto3 huggingface_hub 

In [2]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
from pathlib import Path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

In [4]:
s3_prefix = "hf-large-model-djl/meta-llama/Llama-2-13b-chat"
s3_code_prefix = f"{s3_prefix}/code"  # folder within bucket where code artifact will go
s3_model_prefix = f"{s3_prefix}/model"  # folder within bucket where model artifact will go

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

## Download the model snapshot from Hugging Face and upload the model artifacts on Amazon S3

If you intend to download your copy of the model and upload it to a s3 location in your AWS account, please follow the below steps, else you can skip to the next step.

Following Snapshot Download will take around 4 to 6 mins.

In [ ]:
%%time
from huggingface_hub import snapshot_download
from pathlib import Path
import os

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path(".")
local_model_path.mkdir(exist_ok=True)
model_name = 'meta-llama/Llama-2-13b-chat-hf'
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.txt", "*.model", "*.safetensors", "*.bin", "*.chk", "*.pth"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name, 
    cache_dir=local_model_path, 
    allow_patterns=allow_patterns, 
    token='<YOUR_HUGGING_FACE_READ_ACCESS_TOKEN>'
)

Upload files to default S3 bucket and obtain the URI in a variable.

In [ ]:
base_model_s3_uri = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {base_model_s3_uri}")

In [81]:
# Cleanup locally stored model files post S3 upload
!rm -rf {model_download_path}

## Create SageMaker compatible Model artifact,  upload Model to S3 and bring your own inference script.

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or postprocessing of the model's predictions.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - serving.properties.

The tarball is in the following format:

```
code
├──── 
│   └── serving.properties
```

    serving.properties is the configuration file that can be used to configure the model server.


### Create serving.properties file for neuronx

This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -

- `engine`: The runtime engine for DJL to use. The possible values for engine include *Python*, *DeepSpeed*, *FasterTransformer*, and *MPI*. In this case, we set it to *Python*.
- `option.entryPoint`: model serving engine, we will be using *djl_python.transformers_neuronx* for inferentia 2.
- `option.model_id`: The model id of a pretrained model hosted inside a [model repository on huggingface](https://huggingface.co/models) or S3 path to the model artefact. 
- `option.neuron_optimization_level`: Neuron compiler optimization level, e.g., 1 for fast compilation and 3 for best performance
- `option.tensor_parallel_degree`: number of NeuronCores to be used
- `option.load_in_8bit`: enable/distable int8 weight quantization for reducing memory footprint
- `option.n_positions`: maximum sequence length
- `option.dtype`: date type of weight and activation
- `option.model_loading_timeout`: length of time to timeout in seconds

[Amazon EC2 Inf2 Instances](https://aws.amazon.com/ec2/instance-types/inf2/)

Since we are serving the model using deepspeed container, and Llama 2 being a large model used for inference,  we are following the approach of [Large model inference with DeepSpeed and DJL Serving](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-tutorials-deepspeed-djl.html)


In [159]:
!rm -rf chat_llama2_13b_hf
!mkdir -p chat_llama2_13b_hf
model_id = base_model_s3_uri

In [160]:
%%writefile chat_llama2_13b_hf/serving.properties
engine = Python
option.entryPoint=djl_python.transformers_neuronx
option.model_id={{model_id}}
option.batch_size=8
option.neuron_optimize_level=1
option.tensor_parallel_degree=12
option.load_in_8bit=false
option.n_positions=2048
option.dtype=fp16
option.model_loading_timeout=1500

Writing chat_llama2_13b_hf/serving.properties


In [161]:
# we plug in the appropriate model location into our `serving.properties`
template = jinja_env.from_string(Path("chat_llama2_13b_hf/serving.properties").open().read())
Path("chat_llama2_13b_hf/serving.properties").open("w").write(
    template.render(model_id=base_model_s3_uri)
)
!pygmentize chat_llama2_13b_hf/serving.properties | cat -n

     1	engine = Python
     2	option.entryPoint=djl_python.transformers_neuronx
     3	option.model_id=s3://sagemaker-us-west-2-920487201358/hf-large-model-djl/meta-llama/Llama-2-13b-chat/model
     4	option.batch_size=8
     5	option.neuron_optimize_level=1
     6	option.tensor_parallel_degree=12
     7	option.load_in_8bit=false
     8	option.n_positions=2048
     9	option.dtype=fp16
    10	option.model_loading_timeout=1500


Image URI for the DJL container is being used here

In [162]:
instance_type = "ml.inf2.24xlarge"
inference_image_uri = image_uris.retrieve(
    framework="djl-neuronx", region=region, version="0.25.0",instance_type=instance_type
)
inference_image_uri

'763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.25.0-neuronx-sdk2.15.0'

Create the Tarball and then upload to S3 location

In [163]:
!rm model.tar.gz
!tar czvf model.tar.gz chat_llama2_13b_hf

rm: cannot remove 'model.tar.gz': No such file or directory
chat_llama2_13b_hf/
chat_llama2_13b_hf/serving.properties


In [164]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)

In [165]:
s3_code_artifact

's3://sagemaker-us-west-2-920487201358/hf-large-model-djl/meta-llama/Llama-2-13b-chat/code/model.tar.gz'

In [166]:
!rm model.tar.gz

## Deploy Llama 2 13B Chat LMI Model

[Choosing instance types for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-choosing-instance-types.html)

We will proceed with deploying `meta-llama/Llama-2-13b-chat-hf` model on `ml.g5.12xlarge`

Steps to deploy the model to SageMaker Endpoint will be as follows:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.inf2.24xlarge
    
    b) ContainerStartupHealthCheckTimeoutInSeconds is 900 to ensure health check starts after the model is ready    
3. Create the end point using the endpoint config created    


#### Create the Model
Use the image URI for the DJL container and the s3 location to which the tarball was uploaded.

The container downloads the model into the `/tmp` space on the instance because SageMaker maps the `/tmp` to the Amazon Elastic Block Store (Amazon EBS) volume that is mounted when we specify the endpoint creation parameter VolumeSizeInGB. 
It leverages `s5cmd`(https://github.com/peak/s5cmd) which offers a very fast download speed and hence extremely useful when downloading large models.

For instances like p4dn, which come pre-built with the volume instance, we can continue to leverage the `/tmp` on the container. The size of this mount is large enough to hold the model.


In [167]:
from sagemaker.utils import name_from_base
endpoint_name = name_from_base(f"Llama-2-13b-chat-lmi-inf2")

In [168]:
%%time
from sagemaker import Model
model = Model(image_uri=inference_image_uri, model_data=s3_code_artifact, role=role)
model._is_compiled_model = True # let sagemaker know model is compiled as it is done by neuron-cc
model.deploy(initial_instance_count=1,
             instance_type=instance_type,
             container_startup_health_check_timeout=900,
             volume_size=256,
             endpoint_name=endpoint_name)

---------------------------------!CPU times: user 251 ms, sys: 20.1 ms, total: 271 ms
Wall time: 17min 4s


#### While you wait for the endpoint to be created, you can read more about:
- [Deep Learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)
- [Achieve high performance with lowest cost for generative AI inference using AWS Inferentia2 and AWS Trainium on Amazon SageMaker
](https://aws.amazon.com/blogs/machine-learning/achieve-high-performance-with-lowest-cost-for-generative-ai-inference-using-aws-inferentia2-and-aws-trainium-on-amazon-sagemaker/)

We will store the value of the variable endpoint_name to use it in inference notebook.

In [169]:
%store \
endpoint_name \
bucket \
s3_prefix

Stored 'endpoint_name' (str)
Stored 'bucket' (str)
Stored 's3_prefix' (str)


## Inference Llama 2 13B chat deployed on inf2

In [170]:
from sagemaker import serializers

In [171]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer()
)

In [ ]:
prompts = ["Write a polite and professional dad joke",
          "Write a poem on Sliding ice cubes on a pine tree", ]
results = predictor.predict(
    {"inputs": prompts, "parameters": {"max_new_tokens":256, "do_sample":"true"}}
)

In [173]:
import json
for result in json.loads(results):
    generated_text = result['generated_text']
    print(f"{generated_text}\n")

Write a polite and professional dad joke that includes the phrase "the kids are alright"
 and is funny.

Here's a sample:

"Hey, did you hear about the latest parenting trend? It's called 'letting the kids be alright.' It's like raising them, but on a beach towel. You just spread them out, let them do their thing, and then go grab a drink. The kids are alright, man!"

This joke is funny because it plays on the idea of "letting go" of parental responsibilities and allowing kids to freely express themselves, while also referencing the phrase "the kids are alright" to signify that they are doing well. The parenting trend of "letting the kids be alright" is a humorous exaggeration of the idea of letting kids be independent and self-sufficient. The punchline of the joke, "The kids are alright, man!" adds to the humor by using the colloquial phrase "man" to emphasize the message and create a sense of informality and

Write a poem on Sliding ice cubes on a pine tree

Sliding ice cubes on a pi

In [174]:
cleanup=False
if cleanup:
    sess.delete_endpoint(endpoint_name)
    sess.delete_endpoint_config(endpoint_name)
    model.delete_model()

## References:

- [Improve throughput performance of Llama 2 models using Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/improve-throughput-performance-of-llama-2-models-using-amazon-sagemaker/)
- [Improve performance of Falcon models with Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/improve-performance-of-falcon-models-with-amazon-sagemaker/)
- [serving.properties - Configurations and settings](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html)
- [Amazon SageMaker launches a new version of Large Model Inference DLC with TensorRT-LLM support](https://aws.amazon.com/about-aws/whats-new/2023/11/amazon-sagemaker-large-model-inference-dlc-tensorrt-llm-support/)